In [1]:
from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from SLIM_ElasticNet.SLIMElasticNetRecommender import SLIMElasticNetRecommender

from MatrixFactorization.Cython.MatrixFactorization_Cython import MatrixFactorization_BPR_Cython, MatrixFactorization_FunkSVD_Cython
from MatrixFactorization.PureSVD import PureSVDRecommender

from Base.NonPersonalizedRecommender import TopPop, Random

from KNN.UserKNNCFRecommender import UserKNNCFRecommender
from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender

from KNN.ItemKNNScoresHybridRecommender import ItemKNNScoresHybridRecommender
from KNN.ItemKNNScoresHybridRecommender3 import ItemKNNScoresHybridRecommender3

from KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from KNN.ItemKNNSimilarityHybridRecommender3 import ItemKNNSimilarityHybridRecommender3
from KNN.ItemKNNSimilarityHybridRecommender4 import ItemKNNSimilarityHybridRecommender4

#from GraphBased.RP3betaRecommender import RP3betaRecommender
#from GraphBased.P3alphaRecommender import P3alphaRecommender

In [2]:
import traceback, os
import scipy.sparse
import pandas as pd
import numpy as np

URM_train_sequential = scipy.sparse.load_npz('./Matrix/smallMatrixes/URM_train_sequential.npz')
URM_test_sequential = scipy.sparse.load_npz('./Matrix/smallMatrixes/URM_test_sequential.npz')

URM_train_random = scipy.sparse.load_npz('./Matrix/smallMatrixes/URM_train_random.npz')
URM_test_random = scipy.sparse.load_npz('./Matrix/smallMatrixes/URM_test_random.npz')


In [58]:
#sequential
CFrecommender= ItemKNNCFRecommender(URM_train_sequential)
CFrecommender.fit()
UCFrecommender= UserKNNCFRecommender(URM_train_sequential)
UCFrecommender.fit()
result = CFrecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = UCFrecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))

#random
CFrecommender= ItemKNNCFRecommender(URM_train_random)
CFrecommender.fit()
UCFrecommender= UserKNNCFRecommender(URM_train_random)
UCFrecommender.fit()
result = CFrecommender.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))
result = UCFrecommender.evaluateRecommendations(URM_test_random)
print("Recommender MAP is= {}".format(result["MAP"]))

Similarity column 20635 ( 100 % ), 3275.39 column/sec, elapsed time 0.11 min
Similarity column 5000 ( 100 % ), 7254.89 column/sec, elapsed time 0.01 min
SequentialEvaluator: Processed 4893 ( 100.00% ) in 6.37 seconds. Users per second: 768
Recommender MAP is= 0.06319838918566548
SequentialEvaluator: Processed 4893 ( 100.00% ) in 5.22 seconds. Users per second: 938
Recommender MAP is= 0.05920004620676354
Similarity column 20635 ( 100 % ), 3048.44 column/sec, elapsed time 0.11 min
Similarity column 5000 ( 100 % ), 6170.86 column/sec, elapsed time 0.01 min
SequentialEvaluator: Processed 4805 ( 100.00% ) in 5.50 seconds. Users per second: 874
Recommender MAP is= 0.05550808453993985
SequentialEvaluator: Processed 4805 ( 100.00% ) in 4.29 seconds. Users per second: 1120
Recommender MAP is= 0.052381448224258186


In [105]:
import pandas as pd
from numpy import *
import numpy as np
from scipy import sparse


#SPLITTAGGIO DEI DATI

train_df= pd.read_csv('all/train.csv',low_memory = False)
all_userID=train_df.playlist_id.unique()

target_playlist_df=pd.read_csv('all/target_playlists.csv',low_memory = False)
target_userID=target_playlist_df.playlist_id.unique()

sequential_playlist=target_playlist_df.iloc[:5000]
sequential_id = sequential_playlist.playlist_id.values.tolist()
sequential_df=train_df.loc[train_df['playlist_id'].isin(sequential_id)]
sequential_df["prova"]=0.0

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [121]:
for x in sequential_id:
    seq_id=sequential_df.loc[sequential_df['playlist_id']==x]
    size= seq_id.shape[0]
    val=np.geomspace(0.7, 3, num=size, endpoint=True)
    it=0
    for row in seq_id.itertuples():
        sequential_df.at[row.Index,"prova"]=val[it]
        it=it+1
print(sequential_df)

         playlist_id  track_id     prova
162                7     12493  0.700000
163                7     17495  0.730612
164                7     13424  0.762563
165                7      7109  0.795912
166                7     14714  0.830718
167                7     15650  0.867047
168                7     15740  0.904965
169                7       159  0.944540
170                7     19243  0.985847
171                7     17920  1.028960
172                7      6620  1.073958
173                7      1063  1.120924
174                7     13808  1.169944
175                7     20023  1.221108
176                7      5893  1.274509
177                7      3483  1.330246
178                7      1775  1.388420
179                7      7660  1.449138
180                7      7478  1.512511
181                7      5649  1.578656
182                7      1461  1.647693
183                7     18190  1.719750
184                7      2494  1.794958
185             

In [110]:
#Selezione il 20% da inserire in URM_test e lo metto in sequential_subset
sequential_data=sequential_df.loc[sequential_df['playlist_id'].isin(sequential_id)]
n1=round((115553/100)*20)
for pid in sequential_id:
    playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
    start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
    end=start+round((playlistID.shape[0]*50)/100)

    dropping=list(range(start, end))    
    sequential_data=sequential_data.drop(dropping)


sequential_subset = sequential_data.sample(n=n1)



In [111]:
print(sequential_subset)

         playlist_id  track_id     prova
40848           1730      5355  2.482500
158595          6658     16494  2.342857
778413         32277     11435  2.342857
405888         16891      4236  2.115385
1164140        48448       541  2.841379
666490         27703     20481  2.851613
37656           1588      5660  1.850000
127590          5318     16080  2.014286
1059662        44063     13915  2.651515
515827         21453     17737  1.965000
1112627        46316     17239  2.829630
602962         25055      9610  2.195000
508443         21146      7931  2.382927
322471         13427      3982  2.841379
459129         19069       689  2.123810
632470         26326      4729  3.000000
384255         15996     14547  2.435849
463576         19260        70  2.088679
289158         12049     20371  2.488889
1039181        43232      4731  2.469231
215412          8978      1476  1.915094
75645           3183      8275  2.786047
935269         38845      7868  2.835714
769123         3

In [117]:
from numpy import *
    
#--------------SEQUENTIAL---------------------------    
URM_train_sequential=zeros((5000,20635))
URM_test_sequential=zeros((5000,20635))

for row in sequential_subset.itertuples():
    riga = sequential_id.index(row.playlist_id)
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_test_sequential[riga,colonna]=1

for row in sequential_df.itertuples():
    #print(riga)
    riga= sequential_id.index(row.playlist_id)
    colonna= row.track_id
    #print(colonna)
    URM_train_sequential[riga,colonna]=row.prova
    
for row in sequential_subset.itertuples():
    riga = sequential_id.index(row.playlist_id)
    #print(riga)
    colonna= row.track_id
    #print(colonna)
    URM_train_sequential[riga,colonna]=0
    
URM_train_sequential=sparse.csr_matrix(URM_train_sequential)
URM_test_sequential=sparse.csr_matrix(URM_test_sequential) 

In [118]:
#sequential
CFrecommender= ItemKNNCFRecommender(URM_train_sequential)
CFrecommender.fit()
UCFrecommender= UserKNNCFRecommender(URM_train_sequential)
UCFrecommender.fit()
result = CFrecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))
result = UCFrecommender.evaluateRecommendations(URM_test_sequential)
print("Recommender MAP is= {}".format(result["MAP"]))

Similarity column 20635 ( 100 % ), 6554.51 column/sec, elapsed time 0.05 min
Similarity column 5000 ( 100 % ), 11663.27 column/sec, elapsed time 0.01 min
SequentialEvaluator: Processed 4882 ( 100.00% ) in 3.06 seconds. Users per second: 1598
Recommender MAP is= 0.056570772808190055
SequentialEvaluator: Processed 4882 ( 100.00% ) in 2.69 seconds. Users per second: 1815
Recommender MAP is= 0.055651087251475356


In [ ]:
#linearspace
alphaList=[(1,3),(1,5),(0.7,3),(0.7,5),(0.5,3)]

for aa,bb in alphaList:
    for x in sequential_id:
        seq_id=sequential_df.loc[sequential_df['playlist_id']==x]
        size= seq_id.shape[0]
        val=np.linspace(aa, bb, num=size, endpoint=True)
        it=0
        for row in seq_id.itertuples():
            sequential_df.at[row.Index,"prova"]=val[it]
            it=it+1
    #Selezione il 20% da inserire in URM_test e lo metto in sequential_subset
    sequential_data=sequential_df.loc[sequential_df['playlist_id'].isin(sequential_id)]
    n1=round((115553/100)*20)
    for pid in sequential_id:
        playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
        start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
        end=start+round((playlistID.shape[0]*50)/100)

        dropping=list(range(start, end))    
        sequential_data=sequential_data.drop(dropping)


    sequential_subset = sequential_data.sample(n=n1)



    from numpy import *

    #--------------SEQUENTIAL---------------------------    
    URM_train_sequential=zeros((5000,20635))
    URM_test_sequential=zeros((5000,20635))

    for row in sequential_subset.itertuples():
        riga = sequential_id.index(row.playlist_id)
        #print(riga)
        colonna= row.track_id
        #print(colonna)
        URM_test_sequential[riga,colonna]=1

    for row in sequential_df.itertuples():
        #print(riga)
        riga= sequential_id.index(row.playlist_id)
        colonna= row.track_id
        #print(colonna)
        URM_train_sequential[riga,colonna]=row.prova

    for row in sequential_subset.itertuples():
        riga = sequential_id.index(row.playlist_id)
        #print(riga)
        colonna= row.track_id
        #print(colonna)
        URM_train_sequential[riga,colonna]=0

    URM_train_sequential=sparse.csr_matrix(URM_train_sequential)
    URM_test_sequential=sparse.csr_matrix(URM_test_sequential) 
    #sequential
    CFrecommender= ItemKNNCFRecommender(URM_train_sequential)
    CFrecommender.fit()
    UCFrecommender= UserKNNCFRecommender(URM_train_sequential)
    UCFrecommender.fit()
    result = CFrecommender.evaluateRecommendations(URM_test_sequential)
    print("Recommender MAP is= {}".format(result["MAP"]))
    result = UCFrecommender.evaluateRecommendations(URM_test_sequential)
    print("Recommender MAP is= {}".format(result["MAP"]))

In [123]:
#geomspaced
alphaList=[(1,3),(1,5),(0.7,3),(0.7,5),(0.5,3)]

for aa,bb in alphaList:
    for x in sequential_id:
        seq_id=sequential_df.loc[sequential_df['playlist_id']==x]
        size= seq_id.shape[0]
        val=np.geomspace(aa, bb, num=size, endpoint=True)
        it=0
        for row in seq_id.itertuples():
            sequential_df.at[row.Index,"prova"]=val[it]
            it=it+1
    #Selezione il 20% da inserire in URM_test e lo metto in sequential_subset
    sequential_data=sequential_df.loc[sequential_df['playlist_id'].isin(sequential_id)]
    n1=round((115553/100)*20)
    for pid in sequential_id:
        playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
        start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
        end=start+round((playlistID.shape[0]*50)/100)

        dropping=list(range(start, end))    
        sequential_data=sequential_data.drop(dropping)


    sequential_subset = sequential_data.sample(n=n1)



    from numpy import *

    #--------------SEQUENTIAL---------------------------    
    URM_train_sequential=zeros((5000,20635))
    URM_test_sequential=zeros((5000,20635))

    for row in sequential_subset.itertuples():
        riga = sequential_id.index(row.playlist_id)
        #print(riga)
        colonna= row.track_id
        #print(colonna)
        URM_test_sequential[riga,colonna]=1

    for row in sequential_df.itertuples():
        #print(riga)
        riga= sequential_id.index(row.playlist_id)
        colonna= row.track_id
        #print(colonna)
        URM_train_sequential[riga,colonna]=row.prova

    for row in sequential_subset.itertuples():
        riga = sequential_id.index(row.playlist_id)
        #print(riga)
        colonna= row.track_id
        #print(colonna)
        URM_train_sequential[riga,colonna]=0

    URM_train_sequential=sparse.csr_matrix(URM_train_sequential)
    URM_test_sequential=sparse.csr_matrix(URM_test_sequential) 
    #sequential
    CFrecommender= ItemKNNCFRecommender(URM_train_sequential)
    CFrecommender.fit()
    UCFrecommender= UserKNNCFRecommender(URM_train_sequential)
    UCFrecommender.fit()
    result = CFrecommender.evaluateRecommendations(URM_test_sequential)
    print("Recommender MAP is= {}".format(result["MAP"]))
    result = UCFrecommender.evaluateRecommendations(URM_test_sequential)
    print("Recommender MAP is= {}".format(result["MAP"]))

Similarity column 20635 ( 100 % ), 6228.82 column/sec, elapsed time 0.06 min
Similarity column 5000 ( 100 % ), 12225.33 column/sec, elapsed time 0.01 min
SequentialEvaluator: Processed 4844 ( 100.00% ) in 3.17 seconds. Users per second: 1527
Recommender MAP is= 0.059322358693530794
SequentialEvaluator: Processed 4844 ( 100.00% ) in 2.70 seconds. Users per second: 1797
Recommender MAP is= 0.05612057890486513
Similarity column 20635 ( 100 % ), 5938.88 column/sec, elapsed time 0.06 min
Similarity column 5000 ( 100 % ), 11582.19 column/sec, elapsed time 0.01 min
SequentialEvaluator: Processed 4861 ( 100.00% ) in 3.43 seconds. Users per second: 1417
Recommender MAP is= 0.05032957517163062
SequentialEvaluator: Processed 4861 ( 100.00% ) in 3.00 seconds. Users per second: 1622
Recommender MAP is= 0.054934048346760946
Similarity column 20635 ( 100 % ), 6061.52 column/sec, elapsed time 0.06 min
Similarity column 5000 ( 100 % ), 11812.17 column/sec, elapsed time 0.01 min
SequentialEvaluator: Pro

In [ ]:
#logspace
alphaList=[(1,3),(1,5),(0.7,3),(0.7,5),(0.5,3)]

for aa,bb in alphaList:
    for x in sequential_id:
        seq_id=sequential_df.loc[sequential_df['playlist_id']==x]
        size= seq_id.shape[0]
        val=np.logspace(aa, bb, num=size, endpoint=True)
        it=0
        for row in seq_id.itertuples():
            sequential_df.at[row.Index,"prova"]=val[it]
            it=it+1
    #Selezione il 20% da inserire in URM_test e lo metto in sequential_subset
    sequential_data=sequential_df.loc[sequential_df['playlist_id'].isin(sequential_id)]
    n1=round((115553/100)*20)
    for pid in sequential_id:
        playlistID=sequential_data.loc[sequential_data['playlist_id']==pid]
        start=sequential_data.index[sequential_data['playlist_id'] == pid][0]
        end=start+round((playlistID.shape[0]*50)/100)

        dropping=list(range(start, end))    
        sequential_data=sequential_data.drop(dropping)


    sequential_subset = sequential_data.sample(n=n1)



    from numpy import *

    #--------------SEQUENTIAL---------------------------    
    URM_train_sequential=zeros((5000,20635))
    URM_test_sequential=zeros((5000,20635))

    for row in sequential_subset.itertuples():
        riga = sequential_id.index(row.playlist_id)
        #print(riga)
        colonna= row.track_id
        #print(colonna)
        URM_test_sequential[riga,colonna]=1

    for row in sequential_df.itertuples():
        #print(riga)
        riga= sequential_id.index(row.playlist_id)
        colonna= row.track_id
        #print(colonna)
        URM_train_sequential[riga,colonna]=row.prova

    for row in sequential_subset.itertuples():
        riga = sequential_id.index(row.playlist_id)
        #print(riga)
        colonna= row.track_id
        #print(colonna)
        URM_train_sequential[riga,colonna]=0

    URM_train_sequential=sparse.csr_matrix(URM_train_sequential)
    URM_test_sequential=sparse.csr_matrix(URM_test_sequential) 
    #sequential
    CFrecommender= ItemKNNCFRecommender(URM_train_sequential)
    CFrecommender.fit()
    UCFrecommender= UserKNNCFRecommender(URM_train_sequential)
    UCFrecommender.fit()
    result = CFrecommender.evaluateRecommendations(URM_test_sequential)
    print("Recommender MAP is= {}".format(result["MAP"]))
    result = UCFrecommender.evaluateRecommendations(URM_test_sequential)
    print("Recommender MAP is= {}".format(result["MAP"]))